In [10]:
import numpy as np
import random
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# Load dataset
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- GA Hyperparameter Optimization for Ridge ---
POP_SIZE = 30
GENS = 50
MUT_RATE = 0.3

# Initialize population (alphas between 0.0001 and 100)
population = [10**random.uniform(-4, 2) for _ in range(POP_SIZE)]

def fitness(alpha):
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return -mean_squared_error(y_test, preds)  # maximize fitness (negative MSE)

for gen in range(GENS):
    # Evaluate
    scored = [(alpha, fitness(alpha)) for alpha in population]
    scored.sort(key=lambda x: x[1], reverse=True)
    best = scored[0]
    if gen % 5 == 0:
        print(f"Gen {gen}: best alpha={best[0]:.5f}, MSE={-best[1]:.4f}")

    # Selection (top 20%)
    top = [s[0] for s in scored[:POP_SIZE//5]]

    # New population
    new_pop = []
    while len(new_pop) < POP_SIZE:
        parent = random.choice(top)
        child = parent
        if random.random() < MUT_RATE:
            child = parent * 10**random.uniform(-0.5, 0.5)  # mutate multiplicatively
            child = max(0.0001, min(child, 100))  # keep in range
        new_pop.append(child)
    population = new_pop

# Best GA alpha
best_alpha_ga = best[0]
final_model_ga = Ridge(alpha=best_alpha_ga).fit(X_train, y_train)
mse_ga = mean_squared_error(y_test, final_model_ga.predict(X_test))

print("\n=== GA Results ===")
print(f"Best alpha (GA): {best_alpha_ga:.5f}")
print(f"Test MSE (GA): {mse_ga:.4f}")

# --- GridSearchCV for comparison ---
param_grid = {'alpha': np.logspace(-4, 2, 20)}
grid = GridSearchCV(Ridge(), param_grid, scoring='neg_mean_squared_error', cv=5)
grid.fit(X_train, y_train)

best_alpha_grid = grid.best_params_['alpha']
final_model_grid = Ridge(alpha=best_alpha_grid).fit(X_train, y_train)
mse_grid = mean_squared_error(y_test, final_model_grid.predict(X_test))

print("\n=== GridSearchCV Results ===")
print(f"Best alpha (GridSearchCV): {best_alpha_grid:.5f}")
print(f"Test MSE (GridSearchCV): {mse_grid:.4f}")


Gen 0: best alpha=0.14461, MSE=2853.3857
Gen 5: best alpha=0.15298, MSE=2853.2793
Gen 10: best alpha=0.15775, MSE=2853.2784
Gen 15: best alpha=0.15775, MSE=2853.2784
Gen 20: best alpha=0.15775, MSE=2853.2784
Gen 25: best alpha=0.15569, MSE=2853.2736
Gen 30: best alpha=0.15548, MSE=2853.2736
Gen 35: best alpha=0.15548, MSE=2853.2736
Gen 40: best alpha=0.15548, MSE=2853.2736
Gen 45: best alpha=0.15548, MSE=2853.2736

=== GA Results ===
Best alpha (GA): 0.15548
Test MSE (GA): 2853.2736

=== GridSearchCV Results ===
Best alpha (GridSearchCV): 0.06952
Test MSE (GridSearchCV): 2861.5345
